# Linear Transformation 
We will start with the alignement of the same models for different seeds. 
- First we do not restrict the matrix.
- Second we restrict the matrix to be a rotation matrix.
- Thrid use affine transformation 
- Last but not least we are using different norms and regularization techniques to improve the results



Steps: 
- Load the same model but with different seed
- Sample different images and get latent representation 
- Create Datamatrix X and X'
- Solve the simple optimization problem (Using closed form solution as well cvxpy solver)


In [1]:
# Import relevant libraries
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
import cvxpy as cp

In [2]:
#reimpmort modules with autoreload
%load_ext autoreload
%autoreload 2


In [11]:
# Configuration

config = {
    'path1': "/Users/federicoferoggio/Documents/vs_code/latent-communication/models/checkpoints/VAE/MNIST/MNIST_VAE_1_10.pth",
    'modelname1': 'VAE',
    'seed1': '1',
    'path2': "/Users/federicoferoggio/Documents/vs_code/latent-communication/models/checkpoints/VAE/MNIST/MNIST_VAE_4_10.pth",
    'modelname2': 'VAE',
    'seed2': '1',
    'num_samples': '100',
    'storage_path': 'VAE-ResNet-LinearTransform',

}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")





# Optimization Problem in the Linear Case 
Let $x^i,y^i \in \mathbb{R^n}$ for $i = 1,...,m$ and $A \in \mathbb{R}^{n \times n}$ we are looking for the optimal A, which solves the following optimization problem 
$$ min_A \sum_{i = 1}^n ||Ax^i - y^i||^2 $$
$$ min_ a \sum_{i=1} \sum_{j =1} (A_{(j)} x^i - y^i_j)^2 $$
where we are using the euclidian norm when not otherwise stated.



## Load Model and Transformed Data for VAE
In this section we load the trained models, which we prepaired for our experimental setup

In [13]:
import sys

sys.path.append('..')
from utils.dataloaders.DataLoaderMNIST_single import DataLoader_MNIST


def load_model(model_name, model_path):
    DEVICE = torch.device('cpu')
    
    if model_name == 'VAE':
        from models.definitions.vae import VAE
        model = VAE(in_dim=784, dims=[256, 128, 64, 32], distribution_dim=16).to(DEVICE)
    elif model_name == 'resnet':
        from models.definitions.resnet import ResNet
        model = ResNet().to(DEVICE)
    else:
        raise ValueError(f"Unknown model name: {model_name}")
    
    model.load_state_dict(torch.load(model_path, map_location=DEVICE))
    return model

def load_Models():
    model1 = load_model(config['modelname1'], config['path1'])
    model2 = load_model(config['modelname2'], config['path2'])
    return model1, model2

def get_transformations(model_name):
    if model_name == 'VAE':
        return [
            transforms.ToTensor(),
            transforms.Normalize((0.5,), (0.5,)),
            transforms.Lambda(lambda x: x.view(-1))
        ]
    elif model_name == 'resnet':
        return [
            transforms.ToTensor(),
            transforms.Lambda(lambda x: x.repeat(3, 1, 1))
        ]
    else:
        raise ValueError(f"Unknown model name: {model_name}")

def transformations():
    transformations1 = get_transformations(config['modelname1'])
    transformations2 = get_transformations(config['modelname2'])
    return transformations1, transformations2

# Load models
model1, model2 = load_Models()

# Get transformations
transformations1, transformations2 = transformations()

# Initialize data loader
data_loader_VAE_1 = DataLoader_MNIST(128, get_transformations(config['modelname1']), seed=10)
data_loader_VAE_2 = DataLoader_MNIST(128, get_transformations(config['modelname2']), seed=10)

# Get train and test loaders
train_loader_vae_1 = data_loader_VAE_1.get_train_loader()
test_loader_vae_1 = data_loader_VAE_1.get_test_loader()

train_loader_vae_2 = data_loader_VAE_2.get_train_loader()
test_loader_vae_2 = data_loader_VAE_2.get_test_loader()

# Sample from Dataset

In [16]:
from utils.sampler import simple_sampler, class_sampler

# Convertiere String in Integer
num_samples= int(config['num_samples'])
m = 100

print(num_samples)
# Sample simply 
z1 = simple_sampler(num_samples, model1, model2, data_loader_VAE_1, DEVICE)
z2 = simple_sampler(num_samples, model1, model2, data_loader_VAE_2, DEVICE)

data_loader1, data_loader2 = data_loader.get_train_loader()

# Sample indices from the train set
indices = torch.randperm(len(data_loader1.dataset))[:m]

# Get the corresponding images
all_images_sample1 = torch.stack([data_loader1.dataset[i][0] for i in indices])
all_images_sample2 = torch.stack([data_loader2.dataset[i][0] for i in indices])

z1 = model1.get_latent_space(all_images_sample1.to(DEVICE))
z2 = model2.get_latent_space(all_images_sample2.to(DEVICE))

# Sample from each class
#z1_suf, z2_suf, images_suf, labels_suf = class_sampler(config['num_samples'], model, model2, data_loader, DEVICE)


print(z1.shape)
print(z2.shape)

100


ValueError: too many values to unpack (expected 2)

# Convex Optimization Solver
## Linear Transformation with regularization
We are using the cvxpy solver which is a open source solver for convex optimization problems.

In [ ]:
import os 
#ä Get wd 
print(os.getcwd())
# Linear Transformation
from optimizer import LinearFitting

linear_fitting = LinearFitting(z1, z2,lamda=0.01)

linear_fitting.solve_problem()

loss, A  = linear_fitting.get_results()

name = 'Linear_' + config['modelname1'] + '_' + config['seed1'] + '_' + config['modelname2'] + '_' + config['seed2'] + '_' + config['num_samples']
path = 'optimization/' + config['storage_path'] + '/' + name 

linear_fitting.save_results(path)



## Affine transformation
In this section we implement the affine transformation. We are adding a offset to the problem.

In [ ]:
from optimizer import AffineFitting

affine_fitting = AffineFitting(z1, z2, lamda=0.01)

# Solve the problem
affine_fitting.solve_problem()

# Get the results
loss, A, b = affine_fitting.get_results()

# Show the results
print('Loss:', loss)
print('A:', A)
print('b:', b)

# Save the results
name = 'Affine_' + config['modelname1'] + '_' + config['seed1'] + '_' + config['modelname2'] + '_' + config['seed2'] + '_' + config['num_samples']
path = 'optimization/'+ config['storage_path'] + '/' + name


# Save the results
affine_fitting.save_results(path)



## Linear Transformation with constraints (psd)
In this section we relax the problem and consider that the matrix has to be positive semidefinite.

In [ ]:
# Optimization Variable
A_psd = cp.Variable((32, 32))

# Loss Function
loss_psd = cp.norm2(cp.vstack([A_psd @ z1[i] - z2[i] for i in range(z1.shape[0])]))**2 + lamda * cp.norm(A_psd, 'fro')**2

# Objective Function
objective_psd = cp.Minimize(loss_psd)

# Constraints
constraints = [A_psd >> 0]

# Problem

problem_psd = cp.Problem(objective_psd, constraints)

# Solve the problem

problem_psd.solve()

# Print results
print("Optimal value: ", problem_psd.value)
print(A_psd.value)



## Affine Transfomrmation with constraint (psd)

In [ ]:
A_aff_psd = cp.Variable((32, 32))
b_aff_psd = cp.Variable(32)

# Loss Function
loss_aff_psd = cp.norm2(cp.vstack([A_aff_psd @ z1[i] + b_aff_psd - z2[i] for i in range(z1.shape[0])]))**2 + lamda * cp.norm(A_aff_psd, 'fro')**2

# Objective Function
objective_aff_psd = cp.Minimize(loss_aff_psd)

# Constraints
constraints_aff_psd = [A_aff_psd >> 0]

# Problem
problem_aff_psd = cp.Problem(objective_aff_psd, constraints_aff_psd)

# Solve the problem
problem_aff_psd.solve()

# Print results
print("Optimal value: ", problem_aff_psd.value)
print(A_aff_psd.value)
print(b_aff_psd.value)


## Trying different norms 
We begin to reformulate the problem with the L1 Norm



In [ ]:
A_L1 = cp.Variable((32, 32))

#May need another solver



# Loss Function
loss_L1 = cp.norm1(cp.vstack([A_L1 @ z1[i] - z2[i] for i in range(z1.shape[0])]))**2 + lamda * cp.norm(A_L1, '')**2

# Objective Function
objective_L1 = cp.Minimize(loss_L1)


# Problem
problem_L1= cp.Problem(objective_L1)

# Solve the problem
problem_L1.solve(verbose=True, solver=cp.ECOS)

# Print results
print("Optimal value: ", problem_L1.value)
print(A_L1.value)


# Compare the total distance 

In [ ]:
# Get Latent Space for al